In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier  
import os
import sys
import numpy as np 
from scipy.spatial import distance
from sklearn import preprocessing
from collections import Counter

Reading the csv files 

In [4]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

head() method prints the first five elements of a data frame

In [5]:
print (train_data.head())
y = train_data['Survived']
ID = test_data['PassengerId']

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


concat() concatenates two data frames into one. By default the axis is 0 which conactenates the data frame columnwise while axis 1 concatenates the data frame row wise.

In [6]:
all_data = pd.concat([train_data, test_data])

drop() method removes the entite coulumn

In [7]:
all_data = all_data.drop(['Survived'], axis=1)

fillna fills Nan values in the specified coulumn either with a specified value or with mean, meadian value of that particular column.

In [8]:
all_data['Embarked'] = all_data['Embarked'].fillna('S')
all_data['Fare']  = all_data['Fare'].fillna(all_data['Fare'].median())
all_data['Age']  = all_data['Age'].fillna(all_data['Age'].median())

map function maps certain column value with the ones specified. 

In [9]:
all_data['Embarked'] = all_data['Embarked'].map({'S':0, 'C':1, 'Q':2}).astype(int)
all_data['Sex'] = all_data['Sex'].map({'male':0, 'female':1}).astype(int)

We can also create new features by simply adding two colums and give it a name

all_data['FamilySize'] = all_data['SibSp'].iloc[:] + all_data['Parch'].iloc[:]

In [12]:
all_data.loc[all_data['Fare'] <= 7.91, 'Fare'] = 0
all_data.loc[(all_data['Fare'] > 7.91) & (all_data['Fare'] <= 14.454), 'Fare'] = 1
all_data.loc[(all_data['Fare'] > 14.454) & (all_data['Fare'] <= 31), 'Fare']   = 2
all_data.loc[ all_data['Fare'] > 31, 'Fare']= 3
all_data['Fare'] = all_data['Fare'].astype(int)

all_data.loc[all_data['Age'] <= 16, 'Age'] = 0
all_data.loc[(all_data['Age'] > 16) & (all_data['Age'] <= 32), 'Age'] = 1
all_data.loc[(all_data['Age'] > 32) & (all_data['Age'] <= 64), 'Age'] = 2
all_data.loc[(all_data['Age'] > 64) ,'Age'] = 3
all_data['Age'] = all_data['Age'].astype(int)


Another way to drop columns

In [13]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
all_data = all_data[all_data.columns.difference(drop_elements)]

In [14]:
X = all_data.iloc[:len(train_data)] 
X_test = all_data.iloc[len(train_data):]
min_max_scaler = preprocessing.MinMaxScaler()


splitting the train data into train and validation

In [15]:
def split(X,y, fr):
    split_idx = int(np.ceil(len(X)*fr))
    return (X.iloc[:split_idx], y.iloc[:split_idx], X.iloc[split_idx:], y.iloc[split_idx:])

X_train, y_train, X_val, y_val = split(X, y, 0.8)

Training a classifier on our training data and calculating the accuracy on validation data.

In [17]:
clf = RandomForestClassifier(n_jobs=2)
clf.fit(X_train, y_train,)
y_val_predicted = clf.predict(X_val)
idx = [i for i, j in zip(y_val, y_val_predicted) if i == j]
acc  = (len(idx)/len(y_val))*100
y_test = clf.predict(X_test)
print (acc)

83.70786516853933


Writing to a csv

In [18]:
data = pd.DataFrame({'PassengerId':ID , 'Survived': y_test}, columns=['PassengerId', 'Survived'])
data.set_index(['PassengerId'], inplace = True)
data.to_csv('Submission_rf.csv')